In [1]:
import moveit_commander
import rospy
import ipywidgets as ipw

In [2]:
moveit_commander.roscpp_initialize([])
rospy.init_node('move_group_python_interface_tutorial', anonymous=True)

In [3]:
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
group = moveit_commander.MoveGroupCommander("arm")

In [4]:
planning_grid = ipw.GridspecLayout(3, 3)
planning_grid.grid_gap = '10px'

In [5]:
def get_vbox(description):
    box = ipw.VBox()
    box.layout.height = 'auto'
    inner_box = ipw.VBox()
    inner_box.layout.padding = '5px'
    inner_box.layout.border = '1px solid #CCC'
    inner_box.layout.height = '100%'
    box.children = [ipw.Label(value=description), inner_box]
    return box, inner_box

In [6]:
planning_grid[0:2, 0], commands_box = get_vbox('Commands')

label_commands_box = ipw.Label(value='Commands')
plan_btn = ipw.Button(description='Plan', layout=ipw.Layout(width='auto'))
execute_btn = ipw.Button(description='Execute', layout=ipw.Layout(width='auto'))
plan_execute_btn = ipw.Button(description='Plan and Execute', layout=ipw.Layout(width='auto'))
stop_btn = ipw.Button(description='Stop', layout=ipw.Layout(width='auto'))
label_execution_state = ipw.Label(value='Executed', layout=ipw.Layout(margin='10px', width='auto'))

def on_plan_execute(msg):
    group.set_joint_value_target(group.get_random_joint_values())
    group.plan()
    group.execute()

plan_execute_btn.on_click(on_plan_execute)

commands_box.children = [plan_btn, execute_btn, plan_execute_btn, stop_btn, label_execution_state]

In [7]:
def get_floattext_widget(description, default_value):
    wdg = ipw.FloatText(value=default_value, description=description, layout=ipw.Layout(width='auto'))
    wdg.style.description_width = '75%'
    return wdg

In [8]:
planning_grid[2, 0:2], path_constraints_box = get_vbox('Path Constraints')

constraints_dropdown = ipw.Dropdown(options=['None'], value='None', layout=ipw.Layout(width='auto'))
goal_tolerance_input = get_floattext_widget('Goal Tolerance', 0.0)
path_constraints_box.children = [constraints_dropdown, goal_tolerance_input]

In [9]:
planning_grid[:, 2], options_box = get_vbox('Options')

planning_time_input = get_floattext_widget('Planning Time (s):', 5.0)
planning_attempts_input = get_floattext_widget('Planning Attempts:', 10.0)
velocity_scaling_input = get_floattext_widget('Velocity Scaling:', 1.0)
acceleration_scaling_input = get_floattext_widget('Acceleration Scaling:', 1.0)

def get_bools(opts):
    res = []
    for o in opts:
        ticker = ipw.ToggleButton(description=o)
        ticker.layout.width = 'auto'
        res.append(ticker)
    return res

tickers = get_bools(['Allow Replanning', 'Allow Sensor Positioning', 'Allow External Comm.', 'Use Cartesian Path', 'Use Collision-Aware IK', 'Allow Approx IK Solutions'])

options_box.children = [planning_time_input, planning_attempts_input, velocity_scaling_input, acceleration_scaling_input] + tickers

In [10]:
planning_grid[0:2, 1], query_box = get_vbox('Query')

def get_dropdown_widget(options, description):
    dd = ipw.Dropdown(options=options, value=options[0], description=description, layout=ipw.Layout(width='auto'))
    dd.style.description_width = '50%'
    return dd

constraints_dropdown = get_dropdown_widget(['arm', 'endeffector'], 'Planning Group')
start_state_dropdown = get_dropdown_widget(['<random valid>', '<random>', '<current>', '<same as goal>'], 'Start State')
goal_state_dropdown = get_dropdown_widget(['<random valid>', '<random>', '<current>', '<same as start>'], 'Goal State')
clear_octomap = ipw.Button(description='Clear Octomap')

query_box.children = [constraints_dropdown, start_state_dropdown, goal_state_dropdown, clear_octomap]

In [11]:
constraints_dropdown.description = 'Planning Group'
constraints_dropdown.style.description_width = '50%'

In [12]:
# planning_grid

In [13]:
from moveit_msgs.srv import GetPlannerParams
get_planner_params = rospy.ServiceProxy('/get_planner_params', GetPlannerParams)

def try_float(val):
    try:
        return float(val)
    except:
        return None
    
def try_int(val):
    try:
        return int(val)
    except:
        return None
    
def get_params_box(planner_config, group):
    box = ipw.VBox()
    if planner_config == '<unspecified>':
        return box
    children = []
    params = get_planner_params(planner_config=planner_config, group=group)
    lay = ipw.Layout(width='auto')
        
    for el, val in zip(params.params.keys, params.params.values):
        if try_float(val) is not None:
            children.append(ipw.FloatText(description=el, value=float(val), layout=lay))
        elif try_int(val) is not None:
            children.append(ipw.IntText(description=el, value=int(val), layout=lay))
        else:
            children.append(ipw.Text(description=el, value=val, layout=lay))
    for c in children:
        c.style.description_width = '150px'
    box.children = children
    return box

def get_planner_box(group):
    box = ipw.VBox()
    iface = group.get_interface_description()
    label = ipw.Label(value=iface.name)
    gname = group.get_name()
    ids = iface.planner_ids
    planners = []
    for x in ids:
        if x.startswith(gname) and '[' in x:
            planners.append(x.split('[')[1][:-1])

    param = '/move_group'
    if group.get_name():
        param += '/' + group.get_name()
    param += '/default_planner_config'
    
    default_value = rospy.get_param(param)

    planners.insert(0, '<unspecified>')

    if default_value == 'None':
        default_value = planners[0]

    dropdown = ipw.Dropdown(options=planners, value=default_value, layout=ipw.Layout(width='auto'))
    dropdown.style.description_width = '50%'
    box.children = [label, dropdown]

    subgrid = ipw.GridspecLayout(1, 2)
    subgrid[0, 0] = box
    subgrid[0, 1] = get_params_box(default_value, group.get_name())
    
    def lib_change(msg):
        subgrid[0, 1] = get_params_box(dropdown.value, group.get_name())

    dropdown.observe(lib_change, names='value')
    subgrid.grid_gap = '20px'
    return subgrid

get_planner_box(group)

R3JpZHNwZWNMYXlvdXQoY2hpbGRyZW49KFZCb3goY2hpbGRyZW49KExhYmVsKHZhbHVlPXUnT01QTCcpLCBEcm9wZG93bihsYXlvdXQ9TGF5b3V0KHdpZHRoPXUnYXV0bycpLCBvcHRpb25zPSjigKY=


In [14]:
context_box = ipw.GridspecLayout(1, 1)
context_box.grid_gap = '10px'

In [15]:
context_box[0, 0], planning_library_box = get_vbox('Planning Library')
planning_library_box.children = [get_planner_box(group)]

In [16]:
tab_contents = ['Context', 'Motion Planning']
children = [ipw.Text(description=name) for name in tab_contents]
tab = ipw.Tab()
for i in range(len(children)):
    tab.set_title(i, tab_contents[i])
tab.children = [planning_library_box, planning_grid]

In [17]:
tab

VGFiKGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShHcmlkc3BlY0xheW91dChjaGlsZHJlbj0oVkJveChjaGlsZHJlbj0oTGFiZWwodmFsdWU9dSdPTVBMJyksIERyb3Bkb3duKGxheW91dD1MYXnigKY=
